In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# All imports are defined here:-

In [24]:
%matplotlib inline
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

# Defining Training Directories and CSV's:-

In [25]:
train_dir="/kaggle/input/signature-verification-dataset/sign_data/train"
train_csv="/kaggle/input/signature-verification-dataset/sign_data/train_data.csv"
test_csv="/kaggle/input/signature-verification-dataset/sign_data/test_data.csv"
test_dir="/kaggle/input/signature-verification-dataset/sign_data/test"

In [26]:
df_train=pd.read_csv(train_csv)
df_train.sample(10)

,068/09_068.png,068_forg/03_0113068.PNG,1
2432,004/004_23.PNG,004_forg/0124004_04.png,1
11376,003/003_01.PNG,003/003_18.PNG,0
400,033/12_033.png,033_forg/03_0205033.PNG,1
18489,015/015_09.PNG,015/015_11.PNG,0
16971,017/10_017.png,017_forg/02_0124017.PNG,1
1273,062/05_062.png,062/08_062.png,0
3424,026/05_026.png,026_forg/03_0125026.PNG,1
6448,002/002_24.PNG,002/002_09.PNG,0
10710,003/003_11.PNG,003_forg/0206003_03.png,1
18049,028/12_028.png,028/11_028.png,0


# Here we are seeing that 1 denotes for forged pair and 0 denotes for geniune pair of signatures..

In [27]:
df_test=pd.read_csv(test_csv)
df_test.sample(10)

,068/09_068.png,068_forg/03_0113068.PNG,1
3673,052/05_052.png,052_forg/03_0106052.PNG,1
2696,069/07_069.png,069_forg/01_0111069.PNG,1
1410,051/05_051.png,051/02_051.png,0
1172,067/05_067.png,067/08_067.png,0
5024,064/05_064.png,064/07_064.png,0
1310,051/12_051.png,051_forg/03_0120051.PNG,1
1500,051/09_051.png,051/04_051.png,0
4183,054/11_054.png,054/03_054.png,0
1001,065/09_065.png,065/01_065.png,0
4904,059/04_059.png,059/06_059.png,0


In [28]:
df_train.shape

(23205, 3)

In [29]:
df_test.shape

(5747, 3)

# Making Custom Pytorch Siamese Dataset:-

the ____len____ function which returns the size of the dataset, and

the ____getitem____ function which returns a sample from the dataset given an index.

In [30]:
df_train[4:5]

,068/09_068.png,068_forg/03_0113068.PNG,1
4,068/09_068.png,068_forg/04_0113068.PNG,1


In [31]:
image1_path=os.path.join(train_dir,df_train.iat[4,0])
image1_path

'/kaggle/input/signature-verification-dataset/sign_data/train/068/09_068.png'

In [32]:
class Dataset(Dataset):
    #default constuctor for assigning values
    def __init__(self,train_dir=None,train_csv=None,transform=None):
        self.train_dir=train_dir
        self.train_data=pd.read_csv(train_csv)
        self.train_data.columns=['image1','image2','class']
        self.transform=transform
        
    def __getitem__(self,idx): ## __getitem__ returns a sample data given index, idx=index
        
        img1_path=os.path.join(self.train_dir,self.train_data.iat[idx,0])
        img2_path=os.path.join(self.train_dir,self.train_data.iat[idx,1])
        
        img1=Image.open(img1_path)
        img2=Image.open(img2_path)
        
        img1=img1.convert('L') #L mode image, that means it is a single channel image - normally interpreted as greyscale.
        img2=img2.convert('L')
        
        img1=self.transform(img1)
        img2=self.transform(img2)
        
        return img1, img2, torch.from_numpy(np.array([int(self.train_data.iat[idx,2])],dtype=np.float32))
    
    
    def __len__(self): ## __len__ returns the size of the dataset..
        return len(self.train_data)

# Returns Image1, Image2 and the class label(whether 0 or 1).

In [33]:
# Load the the dataset from raw image folders
dataset = Dataset(train_dir,train_csv,transform=transforms.Compose([transforms.Resize((100,100)),transforms.ToTensor()]))

In [34]:
dataset

In [35]:
# def imshow(img,text=None,should_save=False):
#     npimg = img.numpy()
#     plt.axis("off")
#     if text:
#         plt.text(75, 8, text, style='italic',fontweight='bold',
#             bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()    

# def show_plot(iteration,loss):
#     plt.plot(iteration,loss)
#     plt.show()

In [36]:
# # Viewing the sample of images and to check whether its loading properly
# vis_dataloader = DataLoader(dataset,
#                         shuffle=True,
#                         batch_size=8)
# dataiter = iter(vis_dataloader)


# example_batch = next(dataiter)
# concatenated = torch.cat((example_batch[0],example_batch[1]),0)
# imshow(torchvision.utils.make_grid(concatenated))
# print(example_batch[2].numpy())

# Siamese Network:-

In [37]:
class SiameseNetwork(nn.Module):
    
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        
        
        self.conv1=nn.Conv2d(1,50,kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
         # L1 ImgIn shape=(?, 28, 28, 1)      # (n-f+2*p/s)+1
        #    Conv     -> (?, 24, 24, 50)
        #    Pool     -> (?, 12, 12, 50)
        
        
        self.conv2 = nn.Conv2d(50,60, kernel_size = 5)
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        # L2 ImgIn shape=(?, 12, 12, 50)
        #    Conv      ->(?, 8, 8, 60)
        #    Pool      ->(?, 4, 4, 60)
        
        
        self.conv3 = nn.Conv2d(60, 80,  kernel_size = 3)
        # L3 ImgIn shape=(?, 4, 4, 60)
        #    Conv      ->(?, 2, 2, 80)
       
        
        
        self.batch_norm1 = nn.BatchNorm2d(50)
        self.batch_norm2 = nn.BatchNorm2d(60)
        
#         self.dropout1 = nn.Dropout2d()
        
        # L4 FC 2*2*80 inputs -> 250 outputs
        self.fc1 = nn.Linear(80*2*2, 250) 
        self.fc2 = nn.Linear(250, 2)
        
      
    
    
    def forward1(self,x):
        x=self.conv1(x)
        x = self.batch_norm1(x)
        x=F.relu(x)
        x=self.pool1(x)
        
        x=self.conv2(x)
        x = self.batch_norm2(x)
        x=F.relu(x)
        x=self.pool2(x)
        
        x=self.conv3(x)
        x=F.relu(x)
        
        x = x.view(-1,80*2*2)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        x = F.log_softmax(x, dim=1)
        
        return x
    
    
    def forward(self, input1, input2):
        # forward pass of input 1
        output1 = self.forward1(input1)
        # forward pass of input 2
        output2 = self.forward1(input2)
        
        return output1, output2


# Constrastive Loss Function:-

Contrastive loss:-
Contrastive loss is widely-used in unsupervised and self-supervised learning. Originally developed by Hadsell et al. in 2016 from Yann LeCun’s group, this loss function operates on pairs of samples instead of individual samples. It defines a binary indicator Y for each pair of samples stating whether they should be deemed similar, and a learnable distance function D_W(x_1, x_2) between a pair of samples x_1, x_2, parameterized by the weights W in the neural network. 

, where m>0 is a margin. The margin defines a radius around the embedding space of a sample so that dissimilar pairs of samples only contribute to the contrastive loss function if the distance D_W is within the margin.


Intuitively, this loss function encourages the neural network to learn a embedding to place samples with the same labels close to each other, while distancing the samples with different labels in the embedding space.

In [38]:
class ContrastiveLoss(torch.nn.Module):
    
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [39]:
# Load the dataset as pytorch tensors using dataloader
train_dataloader = DataLoader(dataset,
                        shuffle=True,
                        num_workers=8,
                        batch_size=32)

In [40]:
train_dataloader

In [41]:
# Check whether you have GPU is loaded or not
if torch.cuda.is_available():
    print('Yes')

Yes


In [42]:
net = SiameseNetwork().cuda()
# Loss Function
criterion = ContrastiveLoss()                      # setting criterion
optimizer = torch.optim.SGD(net.parameters(), lr = 3e-4) # setting optimizer
# Declare Optimizer
optimizer = optim.RMSprop(net.parameters(), lr=1e-4, alpha=0.99, eps=1e-8, weight_decay=0.0005, momentum=0.9)

In [43]:
def train():
    counter = []
    loss_history = [] 
    iteration_number= 0
    
    for epoch in range(0,10):
        for i, data in enumerate(train_dataloader,0):
            img0, img1 , label = data
            img0, img1 , label = img0.cuda(), img1.cuda() , label.cuda()
            optimizer.zero_grad()
            output1,output2 = net(img0,img1)
            loss_contrastive = criterion(output1,output2,label)
            loss_contrastive.backward()
            optimizer.step()
            if i %50 == 0 :
                print("Epoch number {}\n Current loss {}\n".format(epoch,loss_contrastive.item()))
                iteration_number +=10
                counter.append(iteration_number)
                loss_history.append(loss_contrastive.item())
    return net

In [44]:
model = train()
torch.save(model.state_dict(), "model.pt")
print("Model Saved Successfully")

Epoch number 0
 Current loss 1.7737867832183838

Epoch number 0
 Current loss 1.2734043598175049

Epoch number 0
 Current loss 1.2776978015899658

Epoch number 0
 Current loss 1.5790718793869019

Epoch number 0
 Current loss 1.4251023530960083

Epoch number 0
 Current loss 1.3061398267745972

Epoch number 0
 Current loss 1.5871816873550415

Epoch number 0
 Current loss 1.439080834388733

Epoch number 0
 Current loss 1.2262494564056396

Epoch number 0
 Current loss 1.2726397514343262

Epoch number 0
 Current loss 1.3761141300201416

Epoch number 0
 Current loss 1.2373507022857666

Epoch number 0
 Current loss 1.255690097808838

Epoch number 0
 Current loss 1.1215015649795532

Epoch number 0
 Current loss 1.1589336395263672

Epoch number 1
 Current loss 1.4345885515213013

Epoch number 1
 Current loss 1.1490474939346313

Epoch number 1
 Current loss 1.3713948726654053

Epoch number 1
 Current loss 1.3765206336975098

Epoch number 1
 Current loss 1.3081318140029907

Epoch number 1
 Curren

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseNetwork().to(device)
model.load_state_dict(torch.load("model.pt"))

<All keys matched successfully>

In [47]:
counter=0
list_0 = torch.FloatTensor([[0]])
list_1 = torch.FloatTensor([[1]])
for i, data in enumerate(test_dataloader,0): 
  x0, x1 , label = data
  concatenated = torch.cat((x0,x1),0)
  output1,output2 = model(x0.to(device),x1.to(device))
  eucledian_distance = F.pairwise_distance(output1, output2)
  if label==list_0:
    label="Orginial"
  else:
    label="Forged"
  imshow(torchvision.utils.make_grid(concatenated),'Dissimilarity: {:.2f} Label: {}'.format(eucledian_distance.item(),label))
  counter=counter+1
  if counter ==20:
     break

NameError: name 'test_dataloader' is not defined